In [1]:
use_dolfinx=True

### colab

In [2]:
# %%px
import os
colab = os.getcwd() == "/content"


In [3]:
if colab:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# drive.flush_and_unmount()  # Smonta Google Drive
# drive.mount('/content/drive', force_remount=True)  # Rimonta


In [5]:
# %%px
if colab:
    if use_dolfinx:
        try:
            import dolfinx
        except ImportError:
            !wget "https://fem-on-colab.github.io/releases/fenicsx-install-release-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
            import dolfinx
        try:
            import gmsh
        except ImportError:
            !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
            import gmsh

--2025-05-09 13:55:52--  https://fem-on-colab.github.io/releases/fenicsx-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4339 (4.2K) [application/x-sh]
Saving to: ‘/tmp/fenicsx-install.sh’

/tmp/fenicsx-instal 100%[===================>]   4.24K  --.-KB/s    in 0s      

2025-05-09 13:55:52 (88.4 MB/s) - ‘/tmp/fenicsx-install.sh’ saved [4339/4339]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICSX_INSTALLED=/usr/local/share/fem-on-colab/fenicsx.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenicsx.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/a51d633e/release

### parallel

In [6]:
## Parallel programming imports
import ipyparallel as ipp
from mpi4py import MPI

In [7]:
os.environ["OMPI_ALLOW_RUN_AS_ROOT"] = "1"
os.environ["OMPI_ALLOW_RUN_AS_ROOT_CONFIRM"] = "1"

### dolfinx

In [8]:

use_dolfinx = True

In [9]:

import os
colab = os.getcwd() == "/content"

In [10]:

if colab:
    if use_dolfinx:
        try:
            import dolfinx
        except ImportError:
            !wget "https://fem-on-colab.github.io/releases/fenicsx-install-release-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
            import dolfinx
        try:
            import gmsh
        except ImportError:
            !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
            import gmsh

### import

In [11]:

from __future__ import print_function


In [12]:
!pip install pyDOE

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyDOE: filename=pydoe-0.3.8-py3-none-any.whl size=18223 sha256=d6bbe12e57fff14e824cddf7ba383ff7d6c49305c3f886be5730951ed849b990
  Stored in directory: /root/.cache/pip/wheels/84/20/8c/8bd43ba42b0b6d39ace1219d6da1576e0dac81b12265c4762e
Successfully built pyDOE


In [13]:

import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys
import copy
import tqdm.autonotebook
if colab:
    if use_dolfinx:
        import gmsh
        from mpi4py import MPI
        from petsc4py import PETSc

        from basix.ufl import element

        from dolfinx.fem import (
            Constant,
            Function,
            functionspace,
            assemble_scalar,
            dirichletbc,
            form,
            locate_dofs_topological,
            locate_dofs_topological,
            set_bc,
        )
        from dolfinx.fem.petsc import (
            apply_lifting,
            assemble_matrix,
            assemble_vector,
            create_vector,
            create_matrix,
            set_bc,
        )
        from dolfinx.geometry import bb_tree, compute_collisions_points, compute_colliding_cells
        from dolfinx.io import VTXWriter, gmshio
        from ufl import (
            FacetNormal,
            Measure,
            TestFunction,
            TrialFunction,
            as_vector,
            div,
            dot,
            dx,
            inner,
            lhs,
            grad,
            nabla_grad,
            rhs,
        )

import pyDOE

  import tqdm.autonotebook



In [14]:

from IPython.display import clear_output as clc

plt.style.use("default")


In [15]:

if colab:
    if use_dolfinx:
        import ufl
        from basix.ufl import element, mixed_element
        from dolfinx import fem, la
        from dolfinx.fem import (Constant, Function, dirichletbc,
                                extract_function_spaces, form, functionspace,
                                locate_dofs_topological)
        from dolfinx.fem.petsc import assemble_matrix_block, assemble_vector_block
        from dolfinx.io import XDMFFile
        from dolfinx.mesh import CellType, create_rectangle, locate_entities_boundary
        from ufl import div, dx, grad, inner
        from ufl import CellDiameter, sqrt, FacetArea

In [16]:

from scipy.interpolate import RBFInterpolator

In [17]:

plt.style.use("default")
%config InlineBackend.figure_format = 'retina'

In [18]:

def add_zoom(zoom = 1.5, dpi = 100):
    plt.gcf().set_figheight(plt.gcf().get_figheight() * zoom)
    plt.gcf().set_figwidth(plt.gcf().get_figwidth() * zoom)
    plt.gcf().dpi = dpi

In [19]:
def scatter_array(arr, label = "label", leg = False, zoom = 1, dpi = 100, s = 1, c = None):
    if not c==None:
        plt.scatter(arr[:, 0], arr[:, 1], label = label, s = s, c = c)
    else:
        plt.scatter(arr[:, 0], arr[:, 1], label = label, s = s)
    # plt.scatter(new_control_points_coords[:, 0], new_control_points_coords[:, 1], c = "b", label = "new control pts")

    add_zoom(zoom, dpi = dpi)
    plt.gca().set_aspect("equal")
    plt.gca().set_adjustable("box")

    if leg:
        plt.gca().legend(loc = "upper right")
    # plt.show()

In [20]:
if colab:
    os.chdir("/content/drive/MyDrive/mySHRED")

### script

In [21]:
# !cat dataset_pinball_parallel.py

In [24]:
# pinball mesh
!gdown https://drive.google.com/uc?id=1glu5FAToGxlJwCZ6TIxLx28C9kNmbtSt

Downloading...
From (original): https://drive.google.com/uc?id=1glu5FAToGxlJwCZ6TIxLx28C9kNmbtSt
From (redirected): https://drive.google.com/uc?id=1glu5FAToGxlJwCZ6TIxLx28C9kNmbtSt&confirm=t&uuid=7f09272c-9365-4098-82f2-49f4797abd22
To: /content/drive/MyDrive/mySHRED/pinball_mesh.msh
100% 672k/672k [00:00<00:00, 89.0MB/s]


In [ ]:
!mpirun -n 8 python dataset_pinball_parallel.py -log_summary -ksp_monitor -pc_gamg_view


Info    : Reading 'pinball_mesh.msh'...
Info    : Reading 'pinball_mesh.msh'...
Info    : 18 entities
Info    : 18 entities
Info    : 7981 nodes
Info    : 7981 nodes
Info    : 15966 elements
Info    : 15966 elements
Info    : Reading 'pinball_mesh.msh'...
Info    : 18 entities
Info    : 7981 nodes
Info    : Reading 'pinball_mesh.msh'...
Info    : 18 entities
Info    : Reading 'pinball_mesh.msh'...
Info    : 7981 nodes
Info    : 18 entities
Info    : 7981 nodes
Info    : Reading 'pinball_mesh.msh'...
Info    : Reading 'pinball_mesh.msh'...
Info    : 18 entities
Info    : 18 entities
Info    : 7981 nodes
Info    : 7981 nodes
Info    : Done reading 'pinball_mesh.msh'
Info    : 15966 elements
Info    : Done reading 'pinball_mesh.msh'
Info    : 15966 elements
Info    : 15966 elements
Info    : 15966 elements
Info    : 15966 elements
Info    : Done reading 'pinball_mesh.msh'
Info    : Done reading 'pinball_mesh.msh'
Info    : Done reading 'pinball_mesh.msh'
Info    : Done reading 'pinball_me